# 🎯 XPLIA - Fonctionnalités Avancées

Ce notebook explore les fonctionnalités avancées de XPLIA:
- Comparaison de modèles
- Différentes méthodes d'explicabilité
- Performance tracking
- Validation et robustesse

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression

from xplia.core import ModelFactory, ExplainerFactory, VisualizerFactory
from xplia.utils import measure_performance, validate_input, validate_model

print("✓ Setup complet")

## 1. Comparaison de Plusieurs Modèles

In [ ]:
# Générer des données
X, y = make_classification(
    n_samples=1000,
    n_features=20,
    n_informative=15,
    n_redundant=5,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"Dataset: {X.shape[0]} samples, {X.shape[1]} features")

In [ ]:
# Entraîner plusieurs modèles
models = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'GradientBoosting': GradientBoostingClassifier(n_estimators=100, random_state=42),
    'LogisticRegression': LogisticRegression(max_iter=1000, random_state=42)
}

results = {}

for name, model in models.items():
    print(f"\n{'='*60}")
    print(f"Modèle: {name}")
    print(f"{'='*60}")
    
    # Entraînement avec mesure de performance
    with measure_performance(f"Training {name}", track_memory=True) as metrics:
        model.fit(X_train, y_train)
    
    # Évaluation
    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)
    
    # Détection et recommandation
    model_type = ModelFactory.detect_model_type(model)
    recommended = ExplainerFactory.get_recommended_method(model)
    
    results[name] = {
        'model': model,
        'train_score': train_score,
        'test_score': test_score,
        'training_time': metrics['elapsed_time'],
        'memory_used': metrics['memory_used'],
        'model_type': model_type,
        'recommended_method': recommended
    }
    
    print(f"Train accuracy: {train_score:.4f}")
    print(f"Test accuracy:  {test_score:.4f}")
    print(f"Training time:  {metrics['elapsed_time']:.3f}s")
    print(f"Memory used:    {metrics['memory_used']:.2f}MB")
    print(f"Model type:     {model_type}")
    print(f"Recommended:    {recommended}")

In [ ]:
# Tableau comparatif
comparison_df = pd.DataFrame([
    {
        'Model': name,
        'Train Acc': f"{r['train_score']:.4f}",
        'Test Acc': f"{r['test_score']:.4f}",
        'Time (s)': f"{r['training_time']:.3f}",
        'Memory (MB)': f"{r['memory_used']:.2f}",
        'Recommended Method': r['recommended_method']
    }
    for name, r in results.items()
])

print("\n" + "="*80)
print("COMPARAISON DES MODÈLES")
print("="*80)
print(comparison_df.to_string(index=False))

## 2. Validation et Robustesse

In [ ]:
# Test de validation des entrées
print("Test de validation des entrées:")
print("="*50)

# Différents formats d'entrée
test_inputs = [
    ("NumPy array", np.array([[1, 2, 3], [4, 5, 6]])),
    ("Pandas DataFrame", pd.DataFrame({'a': [1, 2], 'b': [3, 4], 'c': [5, 6]})),
    ("Liste Python", [[1, 2, 3], [4, 5, 6]]),
]

for name, data in test_inputs:
    validated = validate_input(data)
    print(f"✓ {name:20s} -> shape: {validated.shape}")

In [ ]:
# Test de validation des modèles
print("\nTest de validation des modèles:")
print("="*50)

for name, result in results.items():
    model = result['model']
    try:
        validate_model(model, required_methods=['fit', 'predict', 'predict_proba'])
        print(f"✓ {name:20s} - Toutes les méthodes présentes")
    except ValueError as e:
        print(f"✗ {name:20s} - {e}")

## 3. Méthodes d'Explicabilité Disponibles

In [ ]:
# Lister toutes les méthodes
methods = ExplainerFactory.list_available_methods()

print("Méthodes d'explicabilité disponibles:")
print("="*50)
for i, method in enumerate(methods, 1):
    print(f"{i:2d}. {method}")

print(f"\nTotal: {len(methods)} méthodes")

## 4. Types de Visualisations Disponibles

In [ ]:
# Lister tous les types de graphiques
charts = VisualizerFactory.list_available_charts()

print("Types de graphiques disponibles:")
print("="*50)
for i, chart in enumerate(charts, 1):
    print(f"{i:2d}. {chart}")

print(f"\nTotal: {len(charts)} types de graphiques")

In [ ]:
# Recommandations de graphiques par type d'explication
explanation_types = [
    'feature_importance',
    'shap_values',
    'lime_weights',
    'partial_dependence',
    'interaction',
    'counterfactual',
    'uncertainty'
]

print("\nRecommandations de graphiques:")
print("="*50)
for exp_type in explanation_types:
    recommended = VisualizerFactory.get_recommended_chart(exp_type)
    print(f"{exp_type:25s} -> {recommended}")

## 5. Workflow Complet avec le Meilleur Modèle

In [ ]:
# Sélectionner le meilleur modèle (par test accuracy)
best_model_name = max(results.items(), key=lambda x: x[1]['test_score'])[0]
best_model = results[best_model_name]['model']

print(f"Meilleur modèle: {best_model_name}")
print(f"Test accuracy: {results[best_model_name]['test_score']:.4f}")

# Créer l'explainer
recommended_method = results[best_model_name]['recommended_method']
print(f"\nCréation de l'explainer avec méthode: {recommended_method}")

from xplia.core import create_explainer
explainer = create_explainer(best_model, method='unified')
print(f"✓ Explainer créé: {type(explainer).__name__}")

In [ ]:
# Générer des explications
with measure_performance("Génération explications") as metrics:
    explanation = explainer.explain(X_test[:10])

print(f"\n✓ Explications générées en {metrics['elapsed_time']:.3f}s")
print(f"  Mémoire utilisée: {metrics['memory_used']:.2f}MB")

# Afficher les top features
print("\nTop 5 Features Importantes:")
print("="*50)
for i, fi in enumerate(explanation.feature_importances[:5], 1):
    print(f"{i}. Feature {fi.feature_name:10s} : {fi.importance_value:.4f}")

## 🎉 Conclusion

Ce notebook a démontré:
- ✅ Comparaison de plusieurs modèles
- ✅ Mesure de performance automatique
- ✅ Validation robuste des entrées
- ✅ Recommandations intelligentes
- ✅ Workflow complet d'explicabilité

### Prochaines étapes:
- Essayer avec vos propres données
- Explorer d'autres méthodes d'explicabilité
- Créer des visualisations personnalisées